lack of no. of points info

In [1]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [2]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [3]:
pip install python-geohash

  Using cached python_geohash-0.8.5-cp39-cp39-linux_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import time
import geohash
import logging
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [5]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [6]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

In [7]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection setup
output_schema_name = 'od_matrix'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

# Function to process data for a single day
def process_day(event_date, country_code, sql_engine):
    try:
        # Read data from the SQL table
        pe_tj_df = sql_engine.read_sql(
            f"""
            SELECT 
                cuebiq_id,
                start_lat,
                start_lng,
                end_lat,
                end_lng,
                duration_minutes,
                length_meters,
                number_of_points
            FROM cuebiq.paas_cda_pe_v3.trajectory_uplevelled
            WHERE 
                event_date = {event_date}
                AND end_country = '{country_code}' 
                AND start_country = '{country_code}' 
            """
        )
        logging.info(f"Executing SQL query for date {event_date}")

        # Encode geohashes
        pe_tj_df['start_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['start_lat'], x['start_lng'], precision=3), axis=1)
        pe_tj_df['end_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['end_lat'], x['end_lng'], precision=3), axis=1)

        # Load cell lists from SQL
        try:
            celllist3 = sql_engine.read_sql(f"SELECT geohash3 AS geohash, no_of_unique_users FROM dedicated.pop_density.pd_{country_code}_{event_date}_agg3")
            geohash_dict3 = celllist3.set_index('geohash')['no_of_unique_users'].to_dict()
        except Exception as e:
            logging.warning(f"Failed to load geohash3 data for date {event_date}: {e}")
            geohash_dict3 = {}

        # Add user numbers to the aggregated data
        aggregated_df3 = pe_tj_df.groupby(['start_geohash3', 'end_geohash3']).agg({
            'cuebiq_id': 'count',
            'duration_minutes': ['mean', 'median', 'std'],
            'length_meters': ['mean', 'median', 'std'],
            'number_of_points': ['mean', 'median', 'std']
        }).reset_index()
        aggregated_df3.columns = ['start_geohash3', 'end_geohash3', 'trip_count', 
                                  'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 
                                  'm_length_m', 'mdn_length_m', 'sd_length_m',
                                  'm_points_no', 'mdn_points_no', 'sd_points_no']
        aggregated_df3['start_geohash_user'] = aggregated_df3['start_geohash3'].map(geohash_dict3)
        aggregated_df3['end_geohash_user'] = aggregated_df3['end_geohash3'].map(geohash_dict3)

        # Filter aggregated data and reorder columns
        filtered_df3 = aggregated_df3.dropna(subset=['start_geohash_user', 'end_geohash_user'])
        filtered_df3 = filtered_df3[['start_geohash3', 'start_geohash_user', 'end_geohash3', 'end_geohash_user', 'trip_count', 
                                     'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 
                                     'm_length_m', 'mdn_length_m', 'sd_length_m',
                                     'm_points_no', 'mdn_points_no', 'sd_points_no']]
        return filtered_df3

    except Exception as e:
        logging.error(f"Error processing data for date {event_date}: {e}")
        return pd.DataFrame()

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            logging.info(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            logging.error(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# Main processing loop
def process_date_range(start_date, end_date, country_code, sql_engine):
    start_time = time.time()  # Record start time before processing loop
        
    current_date = start_date
    while current_date <= end_date:
        event_date = current_date.strftime('%Y%m%d')
        
        filtered_df3 = process_day(event_date, country_code, sql_engine)

        # Insert data into the database
        if not filtered_df3.empty:
            table_name = f"od_{country_code.lower()}_{event_date}_agg3"
            insert_data_with_retry(filtered_df3, table_name, con)
        else:
            logging.info(f"No data to insert for date {event_date}")

        # Move to the next day
        current_date += timedelta(days=1)
    end_time = time.time()  # Record end time after processing loop
    total_time = end_time - start_time
    logging.info(f"Total processing time: {total_time:.2f} seconds")

# Example usage:
process_date_range(datetime(2019, 7, 14), datetime(2019, 7, 14), 'ID', sql_engine)

2024-06-07 21:12:34,655 - INFO - Executing SQL query for date 20190714
2024-06-07 21:12:45,217 - INFO - Inserted data into table od_id_20190714_agg3
2024-06-07 21:12:45,218 - INFO - Total processing time: 36.24 seconds


In [9]:
# For fillin missing date gaps

import pandas as pd
import geohash
from datetime import datetime, timedelta
import logging
from sqlalchemy import create_engine
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection setup
output_schema_name = 'od_matrix'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

# Function to process data for a single day
def process_day(event_date, country_code, sql_engine):
    try:
        # Read data from the SQL table
        pe_tj_df = sql_engine.read_sql(
            f"""
            SELECT 
                cuebiq_id,
                start_lat,
                start_lng,
                end_lat,
                end_lng,
                duration_minutes,
                length_meters
            FROM cuebiq.paas_cda_pe_v3.trajectory_uplevelled
            WHERE 
                event_date = {event_date}
                AND end_country = '{country_code}' 
                AND start_country = '{country_code}' 
            """
        )
        logging.info(f"Executing SQL query for date {event_date}")

        # Encode geohashes
        pe_tj_df['start_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['start_lat'], x['start_lng'], precision=3), axis=1)
        pe_tj_df['end_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['end_lat'], x['end_lng'], precision=3), axis=1)

        # Load cell lists from SQL
        try:
            celllist3 = sql_engine.read_sql(f"SELECT geohash3 AS geohash, no_of_unique_users FROM dedicated.pop_density.pd_{country_code}_{event_date}_agg3")
            geohash_dict3 = celllist3.set_index('geohash')['no_of_unique_users'].to_dict()
        except Exception as e:
            logging.warning(f"Failed to load geohash3 data for date {event_date}: {e}")
            geohash_dict3 = {}

        # Add user numbers to the aggregated data
        aggregated_df3 = pe_tj_df.groupby(['start_geohash3', 'end_geohash3']).agg({
            'cuebiq_id': 'count',
            'duration_minutes': ['mean', 'median', 'std'],
            'length_meters': ['mean', 'median', 'std']
        }).reset_index()
        aggregated_df3.columns = ['start_geohash3', 'end_geohash3', 'trip_count', 'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 'm_length_m', 'mdn_length_m', 'sd_length_m']
        aggregated_df3['start_geohash_user'] = aggregated_df3['start_geohash3'].map(geohash_dict3)
        aggregated_df3['end_geohash_user'] = aggregated_df3['end_geohash3'].map(geohash_dict3)

        # Filter aggregated data and reorder columns
        filtered_df3 = aggregated_df3.dropna(subset=['start_geohash_user', 'end_geohash_user'])
        filtered_df3 = filtered_df3[['start_geohash3', 'start_geohash_user', 'end_geohash3', 'end_geohash_user', 'trip_count', 'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 'm_length_m', 'mdn_length_m', 'sd_length_m']]
        return filtered_df3

    except Exception as e:
        logging.error(f"Error processing data for date {event_date}: {e}")
        return pd.DataFrame()

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            logging.info(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            logging.error(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# Main processing loop for a list of dates
def process_date_list(date_list, country_code, sql_engine):
    start_time = time.time()  # Record start time before processing loop

    for event_date in date_list:
        filtered_df3 = process_day(event_date, country_code, sql_engine)

        # Insert data into the database
        if not filtered_df3.empty:
            # table_name = f"{output_schema_name}.{country_code}_agg3"
            table_name = f"od_{country_code.lower()}_{event_date}_agg3"
            insert_data_with_retry(filtered_df3, table_name, con)
        else:
            logging.info(f"No data to insert for date {event_date}")

    end_time = time.time()  # Record end time after processing loop
    total_time = end_time - start_time
    logging.info(f"Total processing time: {total_time:.2f} seconds")

# Example usage with list of specific dates:
date_list = ['20190719', '20190731', '20190808', '20190810', '20190817', '20191020', '20191021']
process_date_list(date_list, 'ID', sql_engine)

2024-06-07 21:25:42,437 - INFO - Executing SQL query for date 20190719
2024-06-07 21:25:51,198 - INFO - Inserted data into table od_id_20190719_agg3
2024-06-07 21:26:03,651 - INFO - Executing SQL query for date 20190731
2024-06-07 21:26:12,308 - INFO - Inserted data into table od_id_20190731_agg3
2024-06-07 21:26:24,771 - INFO - Executing SQL query for date 20190808
2024-06-07 21:26:33,305 - INFO - Inserted data into table od_id_20190808_agg3
2024-06-07 21:26:44,847 - INFO - Executing SQL query for date 20190810
2024-06-07 21:26:52,437 - INFO - Inserted data into table od_id_20190810_agg3
2024-06-07 21:27:05,505 - INFO - Executing SQL query for date 20190817
2024-06-07 21:27:14,620 - INFO - Inserted data into table od_id_20190817_agg3
2024-06-07 21:27:28,089 - INFO - Executing SQL query for date 20191020
2024-06-07 21:27:37,418 - INFO - Inserted data into table od_id_20191020_agg3
2024-06-07 21:27:50,388 - INFO - Executing SQL query for date 20191021
2024-06-07 21:28:00,439 - INFO - In